In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
import mlflow

In [2]:
df=pd.read_csv("cleaned_resale_flat_data.csv")

In [3]:
df=df.copy()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933930 entries, 0 to 933929
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   year                 933930 non-null  int64  
 1   month                933930 non-null  int64  
 2   town                 933930 non-null  object 
 3   flat_type            933930 non-null  object 
 4   block                933930 non-null  object 
 5   street_name          933930 non-null  object 
 6   storey_range         933930 non-null  object 
 7   floor_area_sqm       933930 non-null  float64
 8   flat_model           933930 non-null  object 
 9   lease_commence_date  933930 non-null  float64
 10  resale_price         933930 non-null  float64
 11  remaining_lease      933930 non-null  int64  
dtypes: float64(3), int64(3), object(6)
memory usage: 85.5+ MB


In [5]:
df.shape

(933930, 12)

In [6]:
X=df.drop(columns=["resale_price"])
y=df["resale_price"]

In [7]:
X["flat_age"]=X["year"]-X["lease_commence_date"]

In [8]:
X["month_sin"]=np.sin(2*np.pi*X["year"]/12)
X["month_cos"]=np.cos(2*np.pi*X["year"]/12)

In [9]:
X=X.drop(columns=["month","block","lease_commence_date"])

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42,shuffle=True)

In [11]:
num_features=[
    "year",
    "floor_area_sqm",
    "flat_age",
    "month_sin",
    "month_cos"
]

cat_features=["town","street_name","flat_type","flat_model","storey_range"]

preprocessor=ColumnTransformer(
    transformers=[
        ("num",StandardScaler(), num_features),
        ("cat",OneHotEncoder(handle_unknown="ignore",sparse_output=False), cat_features)
    ],
    n_jobs=-1
)

In [12]:
models={
    "LinearRegression":LinearRegression(),
    "HistGradientBoosting":HistGradientBoostingRegressor(
        learning_rate=0.05,
        max_depth=8,
        max_iter=300,
        random_state=42
    ),
    "XGBoost":XGBRegressor(
        n_estimators=400,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42
    )
}

In [13]:
mlflow.set_experiment("Resale Price Models")

2026/02/11 20:15:28 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/11 20:15:28 INFO mlflow.store.db.utils: Updating database tables
2026/02/11 20:15:28 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/11 20:15:28 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/11 20:15:28 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/11 20:15:28 INFO alembic.runtime.migration: Will assume non-transactional DDL.


<Experiment: artifact_location='file:c:/Users/jk11b/Desktop/flat_price/mlruns/1', creation_time=1770817307684, experiment_id='1', last_update_time=1770817307684, lifecycle_stage='active', name='Resale Price Models', tags={}>

In [14]:
results={}

for name, model in models.items():

    with mlflow.start_run(run_name=name):

        pipeline=Pipeline(
            steps=[
                ("preprocessor",preprocessor),
                ("model",model)
        
            ]
            )
        
        pipeline.fit(X_train,y_train)

        y_pred=pipeline.predict(X_test)

        r2=r2_score(y_test,y_pred)
        mae=mean_absolute_error(y_test,y_pred)
        rmse=mean_squared_error(y_test,y_pred)**0.5

        results[name] = {
        "pipeline": pipeline,
        "R2": r2,
        "MAE": mae,
        "RMSE": rmse,
        }

        #log params
        mlflow.log_param("model",name)

        #log model specific params
        if name!="LinearRegression":
            mlflow.log_params(model.get_params())

        #log metrics
        mlflow.log_metric("R2",r2)
        mlflow.log_metric("MAE",mae)
        mlflow.log_metric("RMSE",rmse)

        #log model
        if name=="XGBoost":
            booster = model.get_booster()
            mlflow.xgboost.log_model(booster,name="model")
        else:
            mlflow.sklearn.log_model(pipeline,name="model")
        
        print(f"{name} | R2={r2:.4f} | MAE={mae:.0f} | RMSE={rmse:.0f}")

LinearRegression | R2=0.8894 | MAE=45695 | RMSE=60808
HistGradientBoosting | R2=0.9681 | MAE=23285 | RMSE=32639
XGBoost | R2=0.9751 | MAE=20746 | RMSE=28867


In [15]:
best_model_name = min(results, key=lambda k: results[k]["RMSE"])

r2   = results[best_model_name]["R2"]
mae  = results[best_model_name]["MAE"]
rmse = results[best_model_name]["RMSE"]

print("Best model:", best_model_name)
print(f"R2   : {r2:.4f}")
print(f"MAE  : {mae:.0f}")
print(f"RMSE : {rmse:.0f}")

Best model: XGBoost
R2   : 0.9751
MAE  : 20746
RMSE : 28867


In [16]:
import joblib
best_pipeline = results[best_model_name]["pipeline"]
joblib.dump(best_pipeline,"backend/best_pipeline.pkl")


['backend/best_pipeline.pkl']

In [17]:
print("Train R2:", pipeline.score(X_train, y_train))
print("Test  R2:", pipeline.score(X_test, y_test))

Train R2: 0.9761770686209624
Test  R2: 0.9750655031037877


In [18]:
from sklearn.utils import shuffle
from sklearn.metrics import r2_score

y_train_shuffled = shuffle(y_train, random_state=42)

pipeline.fit(X_train, y_train_shuffled)
y_pred_fake = pipeline.predict(X_test)

print("Shuffled Test R2:", r2_score(y_test, y_pred_fake))


Shuffled Test R2: 0.002074457031550847


In [19]:
import joblib

In [20]:
from natsort import natsorted

unique_values={
    "town":sorted(df["town"].unique()),
    "street_name":natsorted(df["street_name"].unique()),
    "flat_type":sorted(df["flat_type"].unique()),
    "flat_model":sorted(df["flat_model"].unique()),
    "storey_range":natsorted(df["storey_range"].unique())
}
joblib.dump(unique_values,"backend/unique_values.pkl")

['backend/unique_values.pkl']

In [21]:
df.tail()

,year,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
933925,2025,9,YISHUN,EXECUTIVE,877,YISHUN ST 81,10 TO 12,145.0,MAISONETTE,1987.0,980000.0,61
933926,2025,9,YISHUN,EXECUTIVE,834,YISHUN ST 81,04 TO 06,146.0,MAISONETTE,1988.0,990000.0,62
933927,2025,10,YISHUN,EXECUTIVE,834,YISHUN ST 81,07 TO 09,142.0,APARTMENT,1988.0,990000.0,62
933928,2025,5,YISHUN,MULTI-GENERATION,632,YISHUN ST 61,04 TO 06,147.0,MULTI GENERATION,1987.0,945000.0,61
933929,2025,7,YISHUN,MULTI-GENERATION,633,YISHUN ST 61,01 TO 03,171.0,MULTI GENERATION,1987.0,1095000.0,61
